In [ ]:
!pip install tpot

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
# from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd
# df = pd.read_csv("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/SOLUSDT-5m-test-sample-extracted")
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/SOLUSDT-5m-2022-2023-extracted")
df.index = pd.to_datetime(df["Date"])
df.drop(columns = ["Date"],inplace = True)
df.sort_index(inplace = True)
X = df.drop('Reversal', axis=1)
y = df["Reversal"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, mutual_info_classif

def create_lagged_features(df, n_lags):
    new_df = df.copy()
    for col in df.columns:
        for i in range(1, n_lags+1):
            new_df[col + '_lag_' + str(i)] = df[col].shift(i)
    new_df = new_df.dropna()  # Drop rows with NaN values created by shifting
    return new_df

# Load your data into 'df' and set 'Date' as the index if it isn't already
lag = 10
X = df.drop('Reversal', axis=1)
y = df["Reversal"]
# X_lagged = create_lagged_features(X, lag)

# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X_lagged)
# X_scaled = pd.DataFrame(X_scaled, columns=X_lagged.columns)

# standard_scaler = StandardScaler()
# X_scaled_st = standard_scaler.fit_transform(X_scaled)
# X_scaled_st = pd.DataFrame(X_scaled_st, columns=X_lagged.columns)

# selector = SelectKBest(score_func=mutual_info_classif, k=200)
# X_selected = selector.fit_transform(X_scaled_st, y)
# selected_features = pd.DataFrame(X_selected, columns=[X_lagged.columns[i] for i in range(len(X_lagged.columns)) if selector.get_support()[i]])

# X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42,shuffle = False)

In [ ]:
# X = df.drop(['Potential_profit','Reversal'], axis=1)
# y = df["Reversal"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,shuffle = True)

In [ ]:
y[y == 4] = 1

In [ ]:
import xgboost as xgb
import numpy as np
# Setting parameters for XGBoost model
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': 4,  # Number of classes
    'max_depth': None,  # Maximum depth of the tree
    'eta': 0.1,  # Learning rate
    'gamma': 0.1,  # Minimum loss reduction required to make a further partition
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'eval_metric': 'merror',
    'tree_method' : "hist", 'device' : "cuda"# Evaluation metric to use
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Training the model
num_rounds = 500
evals = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain, num_rounds, evals, verbose_eval=True)

# Making predictions
y_pred = model.predict(dtest)

# Calculating accuracy
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


In [ ]:
xgb.train(params, dtrain, 5000, evals, verbose_eval=True)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(dtest)

report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     28105
           1       0.89      0.91      0.90      6373
           2       0.86      0.89      0.87      6041
           3       0.00      0.00      0.00      1486

    accuracy                           0.93     42005
   macro avg       0.68      0.70      0.69     42005
weighted avg       0.90      0.93      0.92     42005



In [ ]:
importance = model.get_score(importance_type='weight')  # 'weight', 'gain', or 'cover'

# Sort importance scores
importance = dict(sorted(importance.items(), key=lambda item: item[1], reverse=True))

print("Feature Importance Scores:", importance)

Feature Importance Scores: {'Reversal_lag_1': 913.0, 'RelativeATR10_Close-WMA_Close_3': 306.0, 'RelativeATR10_Low-MA_Close_3': 284.0, 'RelativeATR10_High-MA_Close_3': 231.0, 'Reversal_lag_2': 220.0, 'RelativeATR10_High-EMA_Close_3': 192.0, 'RelativeATR10_High-WMA_Close_5': 185.0, 'RelativeATR20_Close-WMA_Close_3': 168.0, 'RelativeATR10_Low-EMA_Close_3': 168.0, 'RelativeATR10_Low-TEMA_Close_5_lag_1': 166.0, 'RelativeATR20_High-WMA_Close_5': 152.0, 'RelativeATR10_Low-WMA_Close_3': 137.0, 'RelativeATR10_High-TEMA_Close_5': 136.0, 'RelativeATR20_Low-WMA_Close_5': 118.0, 'RelativeATR20_Low-MA_Close_3': 109.0, 'RelativeATR20_High-TEMA_Close_5': 108.0, 'RelativeATR10_High-WMA_Close_3': 106.0, 'RelativeATR10_Close-MA_Close_3_lag_1': 106.0, 'RelativeATR10_High-MA_Close_5': 100.0, 'RelativeATR10_Low-WMA_Close_5': 98.0, 'RelativeATR20_Close-MA_Close_3': 95.0, 'RelativeATR10_Low-EMA_Close_3_lag_1': 94.0, 'RelativeATR10_High-TEMA_Close_3_lag_1': 90.0, 'RelativeATR20_High-MA_Close_3': 85.0, 'Absolut

In [ ]:
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False)

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense

params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': len(df['Reversal'].unique()),  # Number of classes
    'max_depth': None,  # Maximum depth of the tree
    'eta': 0.0001,  # Learning rate
    'gamma': 0.1,  # Minimum loss reduction required to make a further partition
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'eval_metric': 'merror',
    'tree_method' : "hist", 'device' : "cuda"# Evaluation metric to use
}

# Placeholder for probabilities
xgb_probabilities = np.zeros((len(X), 4))  # 3 classes

# KFold Cross Validation
kf = KFold(n_splits=3, shuffle=False)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Convert data to DMatrix format for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)

    # Train XGBoost model
    xgb_model = xgb.train(params, dtrain, 100)

    # Predict probabilities
    xgb_probs = xgb_model.predict(dtest)
    xgb_probabilities[test_index] = xgb_probs

# Concatenate original features with XGBoost probabilities
X_train_combined = np.concatenate((X_train_nn, xgb_probabilities[:len(X_train_nn)]), axis=1)
X_test_combined = np.concatenate((X_test_nn, xgb_probabilities[len(X_train_nn):]), axis=1)

# Define the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_combined.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

from keras.optimizers import Adam

custom_optimizer = Adam(learning_rate=0.000001)  # Change the learning rate here

# Compile the model with custom optimizer
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using combined features
model.fit(X_train_combined, y_train_nn, epochs=100, batch_size=32, validation_data=(X_test_combined, y_test_nn))


# Now you can use this trained Keras model for further predictions or evaluation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(sampling_strategy={1: 3000, 2: 3000},  # Adjust to desired number of samples
              k_neighbors=5, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Display resampled class distribution
print(f'Resampled class distribution: {Counter(y_resampled)}')

Resampled class distribution: Counter({0: 5194, 2: 3000, 1: 3000, 3: 218})


In [ ]:
model.fit(X_train_combined, y_train_nn, epochs=300, batch_size=32, validation_data=(X_test_combined, y_test_nn))

In [ ]:
model.fit(X_train_combined, y_train_nn, epochs=300, batch_size=32, validation_data=(X_test_combined, y_test_nn))

In [ ]:
model.fit(X_train_combined, y_train_nn, epochs=300, batch_size=32, validation_data=(X_test_combined, y_test_nn))

In [ ]:
model.fit(X_train_combined, y_train_nn, epochs=1800, batch_size=32, validation_data=(X_test_combined, y_test_nn))

In [ ]:
model.save("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/my_model.keras")

In [ ]:
y_pred

array([[0.6882786 , 0.13882509, 0.14619586, 0.02670045],
       [0.6882786 , 0.13882509, 0.14619586, 0.02670045],
       [0.6882786 , 0.13882509, 0.14619586, 0.02670045],
       ...,
       [0.6882786 , 0.13882509, 0.14619586, 0.02670045],
       [0.6882786 , 0.13882509, 0.14619586, 0.02670045],
       [0.6882786 , 0.13882509, 0.14619586, 0.02670045]], dtype=float32)

In [ ]:
from keras.models import load_model

# Load the saved model
model = load_model("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/my_model.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout, Add, Multiply, Activation, BatchNormalization, Concatenate
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model


# Load the pre-trained model
original_model = load_model("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/my_new_model.keras")

# Assuming the original model has the structure: Input -> Dense -> Dense -> Dense -> Output
# We will reuse all but the output layer

# Create a new model reusing all layers except the last one from the original model
base_layers = original_model.layers[:-1]  # all layers up to before the last layer

# Start a new Sequential model
new_model = Sequential()

# Add each layer from the original model except the output
for layer in base_layers:
    new_model.add(layer)

# Now add new layers
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu'))
new_model.add(Dense(64))
new_model.add(BatchNormalization())
new_model.add(Activation('relu')) # Example of adding a new layer
 # Another new layer

new_model.add(Dense(4, activation='softmax'))  # New output layer, ensure this matches your classification task


# Compile the new model
custom_optimizer = Adam(learning_rate=0.00001)  # Change the learning rate here

# Compile the model with custom optimizer
new_model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Summary of the model to verify structure
new_model.summary()

# Now you can fit your model as required

In [ ]:
new_model.fit(X_train_combined, y_train_nn, epochs=200, batch_size=64, validation_data=(X_test_combined, y_test_nn))

In [ ]:
new_model.save("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/advanced_model.keras")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming 'y_train' is your array of labels
# and it's already encoded as integers for a classification task

# Calculate class weights which inversely proportional to class frequencies
classes = np.unique(y_train_nn)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [ ]:
new_model.fit(X_train_combined, y_train_nn, epochs=500, batch_size=64, class_weight=class_weights,validation_data=(X_test_combined, y_test_nn))

In [ ]:
new_model.save("drive/MyDrive/Colab Notebooks/Trading Bot/Data/Crypto/SOL/advanced_model.keras")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(dtest)

report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.72      0.89      0.79      1286
           1       0.54      0.33      0.41       319
           2       0.54      0.30      0.39       296
           3       0.00      0.00      0.00        59

    accuracy                           0.68      1960
   macro avg       0.45      0.38      0.40      1960
weighted avg       0.64      0.68      0.65      1960



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = new_model.predict(X_test_combined)

report = classification_report(y_test_nn, np.argmax(y_pred, axis=1))

print(report)

In [ ]:

def train_xgb(X_train,y_train,X_test,y_test):
  # Setting parameters for XGBoost model
  params = {
      'objective': 'binary:logistic',  # Multiclass classification
      # 'num_class': 2,  # Number of classes
      'max_depth': None,  # Maximum depth of the tree
      'eta': 0.001,  # Learning rate
      'gamma': 0.1,  # Minimum loss reduction required to make a further partition
      'subsample': 1,  # Subsample ratio of the training instances
      'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree
      # 'eval_metric': 'merror',
      'tree_method' : "hist", 'device' : "cuda"# Evaluation metric to use
  }

  dtrain = xgb.DMatrix(X_train, label=y_train)
  dtest = xgb.DMatrix(X_test, label=y_test)

  # Training the model
  num_rounds = 100
  model = xgb.train(params, dtrain, num_rounds)
  # Making predictions
  y_pred = model.predict(dtest)
  # Calculating accuracy
  accuracy = np.mean(y_pred == y_test)
  print("Accuracy:", accuracy)
  return model



In [ ]:
def one_vs_all(y):
    # Get unique class labels
    unique_classes = np.unique(y)
    n_classes = len(unique_classes)

    # Initialize a list to hold the one-vs-all arrays
    one_vs_all_arrays = []

    # Create one-vs-all arrays for each class
    for cls in unique_classes:
        binary_array = np.where(y == cls, 1, 0)
        one_vs_all_arrays.append(binary_array)

    # Convert list to a numpy array of arrays
    return one_vs_all_arrays

In [ ]:
y_train_0,y_train_1,y_train_2,y_train_3 = one_vs_all(y_train)
y_test_0,y_test_1,y_test_2,y_test_3 = one_vs_all(y_test)

In [ ]:
pred = xgb_0.predict(dtest)

In [ ]:
pred

array([0.67974645, 0.6331609 , 0.6763541 , ..., 0.61544514, 0.67135996,
       0.674188  ], dtype=float32)

In [ ]:
def normalize(pred):
  arr_min = pred.min()
  arr_max = pred.max()
  return (pred - arr_min) / (arr_max - arr_min)

In [ ]:
from sklearn.metrics import fbeta_score
test = y_test_1
y_pred = normalize(xgb_1.predict(dtest))
y_pred = [1 if prob >= 0.19 else 0 for prob in y_pred]
# Calculating accuracy
print(fbeta_score(test, y_pred, beta=2))
print(classification_report(test,y_pred))

In [ ]:
xgb_0 = train_xgb(X_train, y_train_0, X_test, y_test_0)
xgb_1 = train_xgb(X_train, y_train_1, X_test, y_test_1)
xgb_2 = train_xgb(X_train, y_train_2, X_test, y_test_2)
xgb_3 = train_xgb(X_train, y_train_3, X_test, y_test_3)

In [ ]:
y_pred_3 = normalize(xgb_3.predict(dtrain))
# y_pred_3 = [1 if prob >= 0.08 else 0 for prob in y_pred_3]

y_pred_2 = normalize(xgb_2.predict(dtrain))
# y_pred_2 = [1 if prob >= 0.3 else 0 for prob in y_pred_2]

y_pred_0 = normalize(xgb_0.predict(dtrain))
# y_pred_0 = [1 if prob >= 0.1 else 0 for prob in y_pred_0]

y_pred_1 = normalize(xgb_1.predict(dtrain))
# y_pred_1 = [1 if prob >= 0.19 else 0 for prob in y_pred_1]

x_train_merged = np.column_stack([y_pred_0, y_pred_1, y_pred_2, y_pred_3])

In [ ]:
y_pred_test_3 = normalize(xgb_3.predict(dtest))
# y_pred_test_3 = [1 if prob >= 0.08 else 0 for prob in y_pred_test_3]

y_pred_test_2 = normalize(xgb_2.predict(dtest))
# y_pred_test_2 = [1 if prob >= 0.3 else 0 for prob in y_pred_test_2]

y_pred_test_0 = normalize(xgb_0.predict(dtest))
# y_pred_test_0 = [1 if prob >= 0.1 else 0 for prob in y_pred_test_0]

y_pred_test_1 = normalize(xgb_1.predict(dtest))
# y_pred_test_1 = [1 if prob >= 0.19 else 0 for prob in y_pred_test_1]

x_test_merged = np.column_stack([y_pred_test_0, y_pred_test_1, y_pred_test_2, y_pred_test_3])

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=x_train_merged.shape[1], activation='silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('silu'))

model.add(Dense(4, activation='softmax'))

custom_optimizer = Adam(learning_rate=0.01)

model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_merged, y_train, epochs=600, batch_size=64,validation_data=(x_test_merged, y_test))

In [ ]:
x_test_merged

In [ ]:
y_pred

array([[0.9280884 , 0.01528744, 0.04442209, 0.012202  ],
       [0.4771374 , 0.00375373, 0.43353835, 0.08557045],
       [0.9280884 , 0.01528744, 0.04442209, 0.012202  ],
       ...,
       [0.06780223, 0.90909   , 0.00176307, 0.02134464],
       [0.9280884 , 0.01528744, 0.04442209, 0.012202  ],
       [0.9280884 , 0.01528744, 0.04442209, 0.012202  ]], dtype=float32)

In [ ]:
from sklearn.metrics import fbeta_score
y_pred = model.predict(x_test_merged)
y_pred = np.argmax(y_pred, axis=1)
# Calculating accuracy
# print(fbeta_score(y_test, y_pred, beta=2))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def apply_scaling_and_normalization(X):
    # Make sure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame.")

    # Initialize scalers
    min_max_scaler = MinMaxScaler()
    standard_scaler = StandardScaler()
    maxabs_scaler = MaxAbsScaler()
    robust_scaler = RobustScaler()

    # Apply transformations
    X_min_max_scaled = pd.DataFrame(min_max_scaler.fit_transform(X), columns=[f"{col}_minmax" for col in X.columns], index=X.index)
    X_standard_scaled = pd.DataFrame(standard_scaler.fit_transform(X), columns=[f"{col}_standard" for col in X.columns], index=X.index)
    X_maxabs_scaled = pd.DataFrame(maxabs_scaler.fit_transform(X), columns=[f"{col}_maxabs" for col in X.columns], index=X.index)
    X_robust_scaled = pd.DataFrame(robust_scaler.fit_transform(X), columns=[f"{col}_robust" for col in X.columns], index=X.index)

    # Concatenate all the transformed dataframes with the original dataframe
    transformed_X = pd.concat([X, X_min_max_scaled, X_standard_scaled, X_maxabs_scaled, X_robust_scaled], axis=1)

    return transformed_X


In [ ]:
scaled_X = apply_scaling_and_normalization(transformed_X)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, PowerTransformer

def signed_sqrt(x):
    """Apply signed sqrt transformation."""
    return np.sqrt(np.abs(x)) * np.sign(x)

def signed_log(x):
    """Apply signed log transformation after offsetting to avoid log(0)."""
    offset = 1  # This offset ensures we do not have log(0)
    return np.log1p(np.abs(x) + offset) * np.sign(x)

def apply_yeo_johnson(x):
    """Apply Yeo-Johnson transformation with custom error handling."""
    try:
        transformer = PowerTransformer(method='yeo-johnson')
        transformed = transformer.fit_transform(x.reshape(-1, 1))
        return transformed.flatten()
    except ValueError:
        print(f"Failed to transform feature with standard parameters, skipping.")
        return x

def complex_feature_transformations(X):
    X = X.select_dtypes(include=[np.number])
    X = X.astype(np.float32)

    sqrt_transformer = FunctionTransformer(signed_sqrt, validate=False)
    log_transformer = FunctionTransformer(signed_log, validate=False)

    # Apply transformations and convert to DataFrame immediately
    X_sqrt = pd.DataFrame(sqrt_transformer.transform(X), columns=X.columns, index=X.index)
    X_log = pd.DataFrame(log_transformer.transform(X), columns=X.columns, index=X.index)
    X_log_sqrt = pd.DataFrame(log_transformer.transform(X_sqrt), columns=X.columns, index=X.index)

    # Create a list of dataframes to concatenate
    dataframes = [X, X_sqrt, X_log, X_log_sqrt]
    suffixes = ['signed_sqrt', 'signed_log', 'signed_log_sqrt']

    # Update suffixes for new columns
    for df, suffix in zip(dataframes[1:], suffixes):
        df.columns = [f"{col}_{suffix}" for col in df.columns]

    # Concatenate all dataframes at once to avoid fragmentation
    X_transformed = pd.concat(dataframes, axis=1)

    return X_transformed

# Usage example
# X = pd.DataFrame(...)  # Your dataframe here
# transformed_X = complex_feature_transformations(X)
# print(transformed_X)


In [ ]:
transformed_X = complex_feature_transformations(X)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

def apply_polynomial_features(X, degree=2):
    """
    Applies polynomial transformations to the numeric features of the dataframe.
    Args:
    X: pd.DataFrame, the input dataframe with numeric features.
    degree: int, the degree of the polynomial features.

    Returns:
    X_poly: pd.DataFrame, the dataframe with original and polynomial features included.
    """
    # Ensure only numeric columns are considered
    numeric_cols = X.select_dtypes(include=[np.number]).columns

    # Initialize the PolynomialFeatures transformer
    poly_transformer = PolynomialFeatures(degree=degree, include_bias=False)

    # Fit and transform the data
    X_poly = poly_transformer.fit_transform(X[numeric_cols])

    # Generate new feature names
    poly_features_names = poly_transformer.get_feature_names_out(input_features=numeric_cols)

    # Create a DataFrame for the polynomial features
    X_poly_df = pd.DataFrame(X_poly, columns=poly_features_names, index=X.index)

    # Concatenate the new polynomial features to the original DataFrame
    X_transformed = pd.concat([X, X_poly_df], axis=1)

    return X_transformed

In [ ]:
transformed_poly_X = apply_polynomial_features(transformed_X)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

def estimate_best_accuracy(X, y, n_estimators=100, cv=5):
    # Initialize the RandomForestClassifier
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    # Use cross-validation to estimate the best accuracy
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

    # Return the mean accuracy from cross-validation
    return np.mean(scores)

X = df.drop("Reversal", axis=1)
# X = X[["volume","quote_asset_volume","num_trades","taker_buy_base_asset_volume","taker_buy_quote_asset_volume"]]
y = df["Reversal"]

# Normalize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Estimate the best accuracy using Random Forest Classifier
best_accuracy = estimate_best_accuracy(X, y, n_estimators=100, cv=5)
print(f"Best Possible Accuracy: {best_accuracy}")


Best Possible Accuracy: 0.6701020408163265
